In [17]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -5, 10
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)

def f(x):
    #Sphere
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    Sum = 0
    for i in range(0, N-1):
        Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [18]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [19]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [20]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [23]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [24]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [26]:
def SimplexLocalSearch(x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest, k

In [29]:
def IteratedSimplex(Epsilon,EpsilonApostrophe,Lambda,M):
    k = 0
    it = 0
    while(k < M):
        x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
        #set_trace()
        x, _k = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
        if k == 0:
            xBest = x
        if( (k + _k)//1000 > it):
            it = (k + _k)//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest))
        k = k + _k
        if f(x) < f(xBest):
            xBest = x
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
    return xBest

In [30]:
Lambda = 1
M = 100000
Epsilon = 10e-1
#set_trace()
EpsilonApostrophe = 0
print(IteratedSimplex(Epsilon,EpsilonApostrophe,Lambda,M))

***NEW BEST VALUE*** iteration:  704 | best found value:  2886.1094614782055
***NEW BEST VALUE*** iteration:  921 | best found value:  0.45882673596369
iteration: 1000 | best found value:  0.45882673596369
iteration: 2000 | best found value:  0.45882673596369
iteration: 3000 | best found value:  0.45882673596369
iteration: 4000 | best found value:  0.45882673596369
iteration: 5000 | best found value:  0.45882673596369
iteration: 6000 | best found value:  0.45882673596369
iteration: 7000 | best found value:  0.45882673596369
iteration: 8000 | best found value:  0.45882673596369
iteration: 9000 | best found value:  0.45882673596369
iteration: 10000 | best found value:  0.45882673596369
iteration: 11000 | best found value:  0.45882673596369
iteration: 12000 | best found value:  0.45882673596369
iteration: 13000 | best found value:  0.45882673596369
iteration: 14000 | best found value:  0.45882673596369
iteration: 15000 | best found value:  0.45882673596369
iteration: 16000 | best found va